In [51]:
import pandas as pd
import numpy as np

In [20]:
# The data files is stored in the data directory under the root directory of this repo. 
data_dir = "../../data/data/dataanalysis/"
# Name of the file being read
case_sheet = "bmc_cases.xlsb"

#Read the file
data = pd.read_excel(data_dir+case_sheet,engine='pyxlsb',header=None)

In [21]:
# Some sheets have a blank row at top and some don't. By reading without header, dropping the blank rows and assigning the top row as header back, we can fix this. 
# If there is a better solution to this, or later if this problem is resolved if we shift to S3 containers, feel free to edit. 

data = data.dropna(how='all')
data.columns = data.iloc[0]
data = data[1:]

In [22]:
# Count all the non-na (non-empty) values in each columns
nvals = data.notna().sum()
print(nvals)

0
Date                                           100487
S No                                            52926
Ward                                           100487
Test ID (ICMR)                                 100473
Patient name                                        0
Age                                            100487
Gender                                         100487
Mobile Number                                       0
Present Address                                     0
Transferred from Ward                            9015
HealthPost Allocation                           74370
Traceable?                                      98535
Out of Mumbai (Yes / No)                        72999
Assign to other ward                             5828
Person / Official from other ward                   0
Duplicate Entry\n(Yes / No)                     66879
Previous Test ID (only if Duplicate is Yes)      3262
Patient Type (Index / Contact)                  70012
Patient Status            

In [81]:
columns_required = ['Date','Age','Gender','Patient Location (Central)','Patient Status (Central)']

df = data[columns_required]

In [82]:
def get_age_bracket(row):
    age_low = int(row['Age']/10)*10
    age_high = age_low+9
    return str(age_low)+"-"+str(age_high)

In [83]:
df['Age Bracket'] = df.apply(lambda x: get_age_bracket(x),axis=1)

/Users/ayushd/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [84]:
df

,Date,Age,Gender,Patient Location (Central),Patient Status (Central),Age Bracket
1,43921,22,F,Home Isolation,Recovered,20-29
2,43925,88,M,Home Isolation,Recovered,80-89
3,43926,22,M,Home Isolation,Recovered,20-29
4,43927,52,M,Hospitalized,Dead,50-59
5,43929,34,M,Home Isolation,Recovered,30-39
...,...,...,...,...,...,...
100483,44033,27,M,NaN,NaN,20-29
100484,44033,28,F,NaN,NaN,20-29
100485,44033,61,F,NaN,NaN,60-69
100486,44033,39,M,NaN,NaN,30-39


In [85]:
def age_gender_distribution(df):
    x1 = df.groupby(['Date','Age Bracket']).size()
    x2 = df.groupby(['Date','Gender']).size()
    return x1,x2

In [86]:
df['Patient Location (Central)'].value_counts()

Home Isolation      63216
Hospitalized         4540
CCC2                 4025
DCH                  3695
DCHC                 3129
Status Awaited       2610
Private Hospital     1474
Jumbo Facility        499
DCh                     6
Jumbo facility          3
Name: Patient Location (Central), dtype: int64

In [90]:
age_dist = {}
gender_dist = {}

an1,an2 = age_gender_distribution(df)
age_dist['All'] = an1
gender_dist['All'] = an2

facility_types = ['Home Isolation','CCC2','DCHC','Hospitalized','Private Hospital','DCH']
for fac in facility_types : 
    df_fac = df[df['Patient Location (Central)'] == fac]
    an1,an2 = age_gender_distribution(df_fac)
    age_dist[fac] = an1
    gender_dist[fac] = an2

df_fatal = df[df['Patient Status (Central)'] == "Dead"]
an1,an2 = age_gender_distribution(df_fatal)
age_dist['Dead'] = an1
gender_dist['Dead'] = an2

In [92]:
age_dist['Dead']

Date   Age Bracket
43903  60-69          1
43910  40-49          1
       50-59          1
43915  60-69          1
43917  60-69          1
                     ..
44018  20-29          1
       30-39          1
       50-59          1
       60-69          5
       80-89          1
Length: 643, dtype: int64